In [1]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(database="202011060_db", user = "postgres", password = "admin", host = "localhost", port = "5432",options="-c search_path=project")

In [2]:
A='floor, room, week, day'
B = A.replace(" ","")
A_list = B.split(',')
A_list

['floor', 'room', 'week', 'day']

In [3]:
#Generate F,S and T table 
#T=original table without aggregate attribute (attributes present in A)(reference table)
#S=table with not null values or null values (all attributes)
#F=whole table
cur = conn.cursor()
table_name = 'data1'

query_F = 'select * from '+table_name
cur.execute(query_F)
ans_F=cur.fetchall()
F = pd.DataFrame(ans_F, columns=['floor','room','week','day','kwh'])

query_T = 'select '+A+ ' from '+table_name
cur.execute(query_T)
ans_T=cur.fetchall()
T = pd.DataFrame(ans_T, columns=['floor','room','week','day'])


query_S = 'select '+A+' from '+table_name+' where kwh is null'
cur.execute(query_S)
ans_S=cur.fetchall()
S = pd.DataFrame(ans_S, columns=['floor','room','week','day'])

cur.close()